## This is the colab notebook which has some ways in which optimal portfolio can be made. The desgin overview of this portfolio is such that we have selected a combinations of 4 distinct portfolios from the set of 50 stocks

-- In the set of 50 stocks we are going to choose a set of 10 stocks in each portfolio

In [1]:
# Firstly we are going to load all the libraries which we need
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import cvxpy as cp
# importing the libraries



-- Now we are going to load the set of the nifty fifty stocks 

In [2]:
nifty50_Stocks = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS", "KOTAKBANK.NS",
    "HINDUNILVR.NS", "SBIN.NS", "LT.NS", "ITC.NS", "AXISBANK.NS", "BAJFINANCE.NS",
    "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ULTRACEMCO.NS", "NESTLEIND.NS", "WIPRO.NS",
    "NTPC.NS", "POWERGRID.NS", "COALINDIA.NS", "TITAN.NS", "TECHM.NS", "GRASIM.NS",
    "BHARTIARTL.NS", "HCLTECH.NS", "DRREDDY.NS", "ADANIENT.NS", "BAJAJFINSV.NS", "TATASTEEL.NS",
    "HDFCLIFE.NS", "SBILIFE.NS", "HEROMOTOCO.NS", "DIVISLAB.NS", "INDUSINDBK.NS", "JSWSTEEL.NS",
    "CIPLA.NS", "EICHERMOT.NS", "UPL.NS", "BPCL.NS", "BRITANNIA.NS", "SHREECEM.NS",
    "ONGC.NS", "APOLLOHOSP.NS", "BAJAJ-AUTO.NS", "M&M.NS", "HINDALCO.NS", "TATACONSUM.NS",
    "TATAMOTORS.NS", "SBICARD.NS"
]
#importing the nifty fifty stocks

# Defining the time for which we need the data for

data_stocks = yf.download(nifty50_Stocks, start="2014-01-01", end="2024-01-01")

print(data_stocks.head())

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  50 of 50 completed

Price            Close                                                        \
Ticker     ADANIENT.NS APOLLOHOSP.NS ASIANPAINT.NS AXISBANK.NS BAJAJ-AUTO.NS   
Date                                                                           
2014-01-01   37.933189    927.892883    457.836548  247.552444   1448.022583   
2014-01-02   36.041824    890.798096    444.827576  243.740128   1434.920166   
2014-01-03   35.053799    911.284058    448.446228  241.556152   1431.521606   
2014-01-06   35.576038    905.301147    449.499786  240.674911   1426.046509   
2014-01-07   34.150459    892.282043    448.858490  236.450699   1427.292358   

Price                                                                         \
Ticker     BAJAJFINSV.NS BAJFINANCE.NS BHARTIARTL.NS    BPCL.NS BRITANNIA.NS   
Date                                                                           
2014-01-01     75.461990    151.039154    286.472473  32.176537   399.551544   
2014-01-02     72.309875    150.933380 

In [3]:
# Now we need to remove the missing data in the stocks 
data_stocks = data_stocks.dropna(axis=1, thresh=len(data_stocks)*0.95)
import random

# These stocks are diversified acorss different sectors to ensure no single sector dominates your profile
-- After this we are going to find the sectors of each of the following stocks which is available in the yfinance library

# What is stocks diversification
-- Stock diversification is the porcess of spreading your investments across different assests to reduce the overeall risks

-- What does Diversity do in the stock selection,it reduces the risk of getting a major loss when the market crashes as you dont have all your eggs in one basket

In [4]:
available_stocks = list(data_stocks .columns)
random.shuffle(available_stocks)


In [5]:
portfolios = {
    f"Portfolio_{i+1}": available_stocks[i*10:(i+1)*10]
    for i in range(4)
}

portfolios
# these are the portfolios we have used

{'Portfolio_1': [('Open', 'BPCL.NS'),
  ('Close', 'NESTLEIND.NS'),
  ('Close', 'JSWSTEEL.NS'),
  ('Volume', 'DIVISLAB.NS'),
  ('Volume', 'BAJAJFINSV.NS'),
  ('High', 'ADANIENT.NS'),
  ('High', 'BRITANNIA.NS'),
  ('High', 'M&M.NS'),
  ('Open', 'ASIANPAINT.NS'),
  ('Low', 'ICICIBANK.NS')],
 'Portfolio_2': [('Low', 'DIVISLAB.NS'),
  ('Close', 'KOTAKBANK.NS'),
  ('Open', 'INDUSINDBK.NS'),
  ('High', 'BPCL.NS'),
  ('Low', 'MARUTI.NS'),
  ('Open', 'INFY.NS'),
  ('Open', 'TATAMOTORS.NS'),
  ('High', 'TITAN.NS'),
  ('Volume', 'TATAMOTORS.NS'),
  ('Volume', 'M&M.NS')],
 'Portfolio_3': [('Open', 'BRITANNIA.NS'),
  ('Open', 'ULTRACEMCO.NS'),
  ('Close', 'BRITANNIA.NS'),
  ('Open', 'NESTLEIND.NS'),
  ('Volume', 'HCLTECH.NS'),
  ('Open', 'HINDALCO.NS'),
  ('Open', 'UPL.NS'),
  ('Low', 'SHREECEM.NS'),
  ('Close', 'POWERGRID.NS'),
  ('High', 'APOLLOHOSP.NS')],
 'Portfolio_4': [('High', 'INDUSINDBK.NS'),
  ('Open', 'DRREDDY.NS'),
  ('High', 'INFY.NS'),
  ('High', 'DRREDDY.NS'),
  ('Open', 'ONGC.NS'),


In [6]:
returns = data_stocks.pct_change().dropna()

In [7]:
# Create a DataFrame to display the selected portfolios
portfolio_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in portfolios.items()]))

print("🎯 Selected Stocks in Each Portfolio:\n")
print(portfolio_df)

🎯 Selected Stocks in Each Portfolio:

               Portfolio_1              Portfolio_2            Portfolio_3  \
0          (Open, BPCL.NS)       (Low, DIVISLAB.NS)   (Open, BRITANNIA.NS)   
1    (Close, NESTLEIND.NS)    (Close, KOTAKBANK.NS)  (Open, ULTRACEMCO.NS)   
2     (Close, JSWSTEEL.NS)    (Open, INDUSINDBK.NS)  (Close, BRITANNIA.NS)   
3    (Volume, DIVISLAB.NS)          (High, BPCL.NS)   (Open, NESTLEIND.NS)   
4  (Volume, BAJAJFINSV.NS)         (Low, MARUTI.NS)   (Volume, HCLTECH.NS)   
5      (High, ADANIENT.NS)          (Open, INFY.NS)    (Open, HINDALCO.NS)   
6     (High, BRITANNIA.NS)    (Open, TATAMOTORS.NS)         (Open, UPL.NS)   
7           (High, M&M.NS)         (High, TITAN.NS)     (Low, SHREECEM.NS)   
8    (Open, ASIANPAINT.NS)  (Volume, TATAMOTORS.NS)  (Close, POWERGRID.NS)   
9      (Low, ICICIBANK.NS)         (Volume, M&M.NS)  (High, APOLLOHOSP.NS)   

               Portfolio_4  
0    (High, INDUSINDBK.NS)  
1       (Open, DRREDDY.NS)  
2          (High

In [8]:
portfolio_returns = pd.DataFrame(index=returns.index)
# now we are going to make the dataframe of the returns in the form of a dataframe

In [9]:
for name, stocks in portfolios.items():
    portfolio_returns[name] = returns[stocks].mean(axis=1)



In [10]:
portfolio_returns
# these are the returns of all the portfolios.



,Portfolio_1,Portfolio_2,Portfolio_3,Portfolio_4
Date,,,,
2014-01-02,0.209946,0.181255,0.040111,0.180675
2014-01-03,-0.128757,0.163522,0.157605,-0.020068
2014-01-06,-0.007229,-0.023373,-0.047996,-0.092703
2014-01-07,-0.051522,0.012049,0.176290,0.103038
2014-01-08,0.109484,-0.001866,-0.036941,0.107710
...,...,...,...,...
2023-12-22,0.193359,0.052907,-0.008460,-0.081339
2023-12-26,0.045157,-0.090501,-0.029288,0.064470
2023-12-27,-0.053678,0.165428,0.009649,0.081105


## After we have taken out the returns of the porfolios , we need to take the last 10 years of data 
From that we are going to Break the period into non-overlapping 20-day windows

In [ ]:
window_results = []

for i in range(0, len(portfolio_returns) - 20 + 1, 20):
    window = portfolio_returns.iloc[i:i+20]

    # after creating a loop we have made

    # Cumulative return of each portfolio over the 20-day window
    cumulative = (1 + window).prod() - 1

    # Market average return per day
    daily_mean = window.mean(axis=1)

    # Volatility of the window
    volatility = daily_mean.std()

    # Total return of the market (mean return of all portfolios)
    total_return = (1 + daily_mean).prod() - 1

    # Label as Crash if volatility is high and returns are sharply negative
    label = 'Crash' if (volatility > 0.015 and total_return < -0.03) else 'Normal'

    result = {
        'Start_Date': window.index[0],
        'Portfolio_1': round(cumulative['Portfolio_1'], 4),
        'Portfolio_2': round(cumulative['Portfolio_2'], 4),
        'Portfolio_3': round(cumulative['Portfolio_3'], 4),
        'Portfolio_4': round(cumulative['Portfolio_4'], 4),
        'Volatility': round(volatility, 4),
        'Market_Regime': label
    }

    window_results.append(result)

# Create DataFrame
windows_df = pd.DataFrame(window_results)
print(windows_df.head(10))

  Start_Date  Portfolio_1  Portfolio_2  Portfolio_3  Portfolio_4  Volatility  \
0 2014-01-02       0.4936       0.8051       0.4874       1.0891      0.0619   
1 2014-01-30       1.6092       0.7200       0.3279       2.2494      0.0680   
2 2014-02-28       1.5537       0.4230       0.2006       2.1468      0.0643   
3 2014-03-31          inf          inf          inf          inf         NaN   
4 2014-05-02       7.1689       1.8016       0.3321       2.1585      0.1237   
5 2014-05-30       1.4545       0.4669       0.2068       1.3464      0.0588   
6 2014-06-27       1.9141       0.3318       0.4435       0.9351      0.0933   
7 2014-07-25       1.9105       0.5267       0.2025       1.6341      0.0495   
8 2014-08-26       2.4668       0.8282       0.6888       0.7696      0.1150   
9 2014-09-24          inf          inf          inf          inf         NaN   

  Market_Regime  
0        Normal  
1        Normal  
2        Normal  
3        Normal  
4        Normal  
5        No

/Users/Lenovo/.venvs/py13env/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/Lenovo/.venvs/py13env/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


## Why have we chosen the given values of the vol_thresh and return_threshold is that duirng the 20 day overlap period which we have taken , a drop of 3% or more is considered in the negative direction.
-- Since we want to capture the crahses as well as the times where the stocks where in the right direction to say , profit
